# Files and versioning

Unless you are a string theorist, at some point you're probably going to want to save and load some data. This tutorial covers some of Sciris' tools for doing that more easily.

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/sciris/sciris/HEAD?labpath=docs%2Ftutorials%2Ftut_files.ipynb) to open an interactive version of this notebook.

</div>


<div class="alert alert-warning">
    <b>Warning!</b> The tools here are powerful, which also makes them dangerous. Unless it's in a simple text format like JSON or CSV, loading a datafile can run arbitrary code on your computer, just like running a Python script can. If you wouldn't run a Python file from some source, don't open a data file from that source either.
</div>

## Saving and loading literally anything

Let's assume you're mostly just saving and loading files you've created yourself or from trusted colleagues, not opening email attachments from the branch of the local mafia. Then everything here is absolutely fine.

Let's revisit our sim from the first tutorial:

In [ ]:
import sciris as sc
import numpy as np
import pylab as pl
sc.options(jupyter=True) # To make plots nicer

class Sim:
    
    def __init__(self, days, trials):
        self.days = days
        self.trials = trials
    
    def run(self):
        self.x = np.arange(self.days)
        self.y = np.cumsum(np.random.randn(self.days, self.trials)**3, axis=0)
    
    def plot(self):
        with pl.style.context('sciris.fancy'):
            pl.plot(self.x, self.y, alpha=0.6)

# Run and save
sim = Sim(days=30, trials=5)
sim.run()
sc.save('my-sim.obj', sim) # Save any Python object to disk

# Load and plot
new_sim = sc.load('my-sim.obj') # Load any Python object
new_sim.plot()

We can create a sim (which of course doesn't have to be a sim, it can be absolutely any object), then reload it from disk and it works just like new – even calling methods works! What's happening here? Under the hood, `sc.save()` saves the object as a [gzipped](https://docs.python.org/3/library/gzip.html) (compressed) [pickle]](https://docs.python.org/3/library/pickle.html) (byte stream). Pickles are how Python sends objects internally, so can handle almost anything. (For the few corner cases that `pickle` can't handle, `sc.save()` falls back on [dill](https://dill.readthedocs.io/en/latest/), which really can handle everything.) 

There are also other compression options than gzip ([zstandard](https://python-zstandard.readthedocs.io/en/latest/) or no compression), but you probably don't need to worry about these. (If you _really_ care about performance, then `sc.zsave()`, which uses `zstandard` by default, is slightly faster than `sc.save()` – but regardless of how a file was saved you can load it with `sc.load()`.

## Saving and loading JSON and YAML

While `sc.save()` and `sc.load()` are great for many things, they _aren't_ great for just sharing data. First, they're not compatible with anything other than Sciris, so if you try to share one of those files with, say, an R user, they won't be able to open them. 

If you just have data and don't want to save the objects, you should save just the data. If you want to save to CSV or Excel (i.e., data that looks like a spreadsheet), you should convert it to a dataframe (`df = sc.dataframe(data)`), then save it from there (`df.to_excel()` and `df.to_csv()`, respectively). 

But if you want to save data that's a little more complex, you should consider JSON: it's fast, it's easy for humans to read, and absolutely everything loads it. While typically a JSON maps onto a Python `dict`, Sciris will take pretty much any object and save out the JSONifiable parts of it:

In [ ]:
# Try saving our sim as a JSON
sc.savejson('my-sim.json', sim)

# Load it as a JSON
sim_json = sc.loadjson('my-sim.json')
print(sim_json)

It's not exactly beautiful, and it's not as powerful as `sc.save()` (for example, `sim_json.plot()` doesn't exist), but it has all the _data_, exactly as it was laid out in the original object:

In [ ]:
print(f"{sim_json['x'] = }")
print(f"{sim_json['y'][0] = }")

(Note that when exported to JSON and loaded back again, everything is in default Python types – so the data is now a list of lists rather than a 2D Numpy array.)

If you're not super familiar with [YAML](https://yaml.org/), you might think of it as that quirky config file format with lots of colons and indents. It _is_ that, but it's also a powerful extension to JSON – every JSON file is also a valid YAML file, but the reverse is not true (i.e., JSON is a subset of YAML). Of most interest to you, dear scientist, is that you can add comments to YAML files. Consider this (relatively) common situation:

In [ ]:
raw_json = '''
{"variables": {
    "timepoints": [0,1,2,3,4,5],
    "really_important_variable": 12.566370614359172
  }
}
'''
data = sc.readjson(raw_json)
print(data)

Now you're tearing your hair out. Where did 12.566370614359172? It looks vaguely familiar, or at least it did to the you from 6 months ago who wrote it. But with YAML, you can have your data and comment it too:

In [ ]:
raw_yaml = '''
{"variables": {
    "timepoints": [0,1,2,3,4,5],
    "really_important_variable": 12.566370614359172 # This is just 4π lol
  }
}
'''
data = sc.readyaml(raw_yaml)
print(data)

## Other file functions

Sciris includes a number of other file utilities. For example, to get a list of files, you can use `sc.getfilelist()`:

In [ ]:
sc.getfilelist('*.ipynb')

Sometimes it's useful to get the folder for the current file, since sometimes you're calling it from a different place, and want the relative paths to remain the same (for example, to load something from a subfolder):

In [ ]:
sc.thispath()

(This looks wonky here because this notebook is run on some random cloud server, but it should look more normal if you do it at home!)

Most Sciris file functions can return either strings or [Paths](https://docs.python.org/3/library/pathlib.html). If you've never used `pathlib`, it's a really powerful way of handling paths. But it's also really intuitive. For example, to create a `data` subfolder that's always relative to this notebook regardless of where it's run from, you can do

In [ ]:
datafolder = sc.thispath() / 'data'
print(datafolder)

(Note: this path might look a little janky because this notebook was run on some cloud server. It will probably look more normal for you!)

Sciris makes it easy to ensure that a path exists:

In [ ]:
datafile = sc.makefilepath(datafolder / 'my-data.csv', makedirs=True)
print(datafile)

Sciris usually handles all this internally, but this can be useful for using with non-Sciris functions, e.g.

In [ ]:
np.savetxt('data/my-data.csv', np.random.rand(2,2)) # Would give an error without sc.makefilepath() above

Lastly, you can clean up with yourself with `sc.rmpath()`, which will automatically figure out whether to use [`os.remove()`](https://docs.python.org/3/library/os.html#os.remove) (which works for files but not folders) or [`shutil.rmtree()`](https://docs.python.org/3/library/shutil.html#shutil.rmtree) (which works for folders but not files):

In [ ]:
sc.rmpath('data/my-data.csv')